### Replica 7: Réplica de RAG

#### A. Indexar documentos con un cleaner

In [2]:
from haystack.dataclasses import Document

documents = [
    Document(
        content="""Munich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural heritage and modern urban sophistication. Nestled along the banks of the Isar River, Munich is renowned for its splendid architecture, including the iconic Neues Rathaus (New Town Hall) at Marienplatz and the grandeur of Nymphenburg Palace. The city is a haven for art enthusiasts, with world-class museums like the Alte Pinakothek housing masterpieces by renowned artists. Munich is also famous for its lively beer gardens, where locals and tourists gather to enjoy the city's famed beers and traditional Bavarian cuisine. The city's annual Oktoberfest celebration, the world's largest beer festival, attracts millions of visitors from around the globe. Beyond its cultural and culinary delights, Munich offers picturesque parks like the English Garden, providing a serene escape within the heart of the bustling metropolis. Visitors are charmed by Munich's warm hospitality, making it a must-visit destination for travelers seeking a taste of both old-world charm and contemporary allure."""
    )
]

In [3]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.document_stores.in_memory import InMemoryDocumentStore

#! 1. Definir componentes del pipeline
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
document_cleaner = DocumentCleaner()
document_splitter = DocumentSplitter(split_length=1, split_overlap=0, split_by="sentence")
document_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
document_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE)

#! 2. Crear pipeline
pipeline = Pipeline()
pipeline.add_component("cleaner", document_cleaner)
pipeline.add_component("splitter", document_splitter)
pipeline.add_component("embedder", document_embedder)
pipeline.add_component("writer", document_writer)

#! 3. Conectar pipeline
pipeline.connect("cleaner", "splitter")
pipeline.connect("splitter", "embedder")
pipeline.connect("embedder", "writer")

🚅 Components
  - cleaner: DocumentCleaner
  - splitter: DocumentSplitter
  - embedder: SentenceTransformersDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - cleaner.documents -> splitter.documents (List[Document])
  - splitter.documents -> embedder.documents (List[Document])
  - embedder.documents -> writer.documents (List[Document])

In [4]:
from pprint import pprint

result = pipeline.run({"cleaner": {"documents": documents}})
pprint(result)

/home/nahumfg/GithubProjects/TesisMaestria/TesisHaystack/venv_haystack_ai/lib/python3.11/site-packages/sentence_transformers/SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'writer': {'documents_written': 7}}


In [5]:
document_store.filter_documents()

[Document(id=af7526f937460d9eb010c476df74c405f82906aa015dac714b193d94d5e67ebe, content: 'Munich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural ...', meta: {'source_id': 'f2d45a0dcb080e4ded436659c9a2c497dd55a46213b1c3903eab752c9ac4ca48', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0}, embedding: vector of size 384),
 Document(id=f2ba80a4c91151abdd8a1895c7ac1bab4651e58ccceaf524319e93dc3341da4b, content: ' Nestled along the banks of the Isar River, Munich is renowned for its splendid architecture, includ...', meta: {'source_id': 'f2d45a0dcb080e4ded436659c9a2c497dd55a46213b1c3903eab752c9ac4ca48', 'page_number': 1, 'split_id': 1, 'split_idx_start': 141}, embedding: vector of size 384),
 Document(id=2f76b2eabc919e4d2fc8145777e724652c94d2d4ce018b584cee898565cce184, content: ' The city is a haven for art enthusiasts, with world-class museums like the Alte Pinakothek housing ...', meta: {'source_id': 'f2d45a0dcb080e4ded436659c9a2c497dd55a462

#### B. RAG pipeline

In [6]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret

#! 1. Definir componentes del pipeline
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
retriever = InMemoryEmbeddingRetriever(document_store=document_store)

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
""" 
prompt_builder = PromptBuilder(template=template)

generator = OpenAIGenerator(
    api_key= Secret.from_env_var("OPENAI_API_KEY"),
    model="gpt-4o-mini-2024-07-18"
) 

In [7]:
from haystack import Pipeline

#! 2. Crear pipeline
rag_pipeline = Pipeline()
rag_pipeline.add_component("text_embedder", text_embedder)
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("generator", generator)

In [11]:

#! 3. Conectar pipeline
rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "generator")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - generator: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)

In [12]:
from pprint import pprint

question = "Where is Munich?"

response = rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})
pprint(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'generator': {'meta': [{'finish_reason': 'stop',
                         'index': 0,
                         'model': 'gpt-4o-mini-2024-07-18',
                         'usage': {'completion_tokens': 26,
                                   'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0),
                                   'prompt_tokens': 235,
                                   'prompt_tokens_details': PromptTokensDetails(audio_tokens=0, cached_tokens=0),
                                   'total_tokens': 261}}],
               'replies': ['Munich is located in southern Germany. It is the '
                           'capital of Bavaria and is situated along the banks '
                           'of the Isar River.']}}
